# Human tracking
Human tracking was achieved by using a SORT algorithm, which is a Simple Online and Real-time Tracking 2D multiple object tracking (MOT) in video frames. Incorporated into the pipeline of the project *Mask detection and social distancing* using the following GitHub repository: </br>
@misc{multiobjtracker_amd2018,
  author = {Deshpande, Aditya M.},
  title = {Multi-object trackers in Python},
  year = {2020},
  publisher = {GitHub},
  journal = {GitHub repository},
  howpublished = {\url{https://github.com/adipandas/multi-object-tracker}},
}

## Dealing with data set
### Easy way of downloading data sets via download link (No need to run the next cell, does not work for now!)

In [ ]:
# using gdown
import gdown
gdown.download("https://www.kaggle.com/akashguna/lfw-dataset-with-masks/download", output="archive.zip", quiet=False)
gdown.download("https://www.kaggle.com/andrewmvd/face-mask-detection/download", output="dataset2.zip", quiet=False)
!unzip /content/dataset2.zip
# !tar xzf /content/aclImdb_v1.tar.gz

# using wget
! wget https://www.kaggle.com/akashguna/lfw-dataset-with-masks/download
! unzip download

### Mounting Google Drive

In [11]:
from google.colab import drive
import glob, os

drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive # creating a symbolic link
os.path.isfile('/mydrive/video2.mp4')

Mounted at /content/gdrive


True

## Using the code for SORT
We clone the GitHub repository and install the necessary dependencies:

In [2]:
!git clone https://github.com/adipandas/multi-object-tracker
%cd multi-object-tracker
!pip install -r requirements.txt
!pip install -e .
!pip install ipyfilechooser

Cloning into 'multi-object-tracker'...
remote: Enumerating objects: 637, done.
remote: Counting objects: 100% (225/225), done.
remote: Compressing objects: 100% (145/145), done.
remote: Total 637 (delta 96), reused 181 (delta 78), pack-reused 412
Receiving objects: 100% (637/637), 87.61 MiB | 27.08 MiB/s, done.
Resolving deltas: 100% (314/314), done.
/content/multi-object-tracker
     |████████████████████████████████| 151 kB 6.9 MB/s 
     |████████████████████████████████| 64 kB 2.6 MB/s 
     |████████████████████████████████| 50 kB 5.8 MB/s 
     |████████████████████████████████| 42 kB 770 kB/s 
     |████████████████████████████████| 69 kB 6.5 MB/s 
     |████████████████████████████████| 280 kB 54.4 MB/s 
     |████████████████████████████████| 99 kB 9.2 MB/s 
     |████████████████████████████████| 280 kB 61.5 MB/s 
     |████████████████████████████████| 280 kB 36.0 MB/s 
     |████████████████████████████████| 280 kB 48.3 MB/s 
     |████████████████████████████████| 279 kB 5

Then, we take the weights and configuration for Yolov3:

In [3]:
# downloading yolo weights
!wget https://pjreddie.com/media/files/yolov3.weights
!wget https://raw.githubusercontent.com/pjreddie/darknet/master/cfg/yolov3.cfg
!wget https://raw.githubusercontent.com/pjreddie/darknet/master/data/coco.names

--2021-11-26 14:06:46--  https://pjreddie.com/media/files/yolov3.weights
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248007048 (237M) [application/octet-stream]
Saving to: ‘yolov3.weights’

yolov3.weights      100%[===================>] 236.52M   104MB/s    in 2.3s    

2021-11-26 14:06:49 (104 MB/s) - ‘yolov3.weights’ saved [248007048/248007048]

--2021-11-26 14:06:49--  https://raw.githubusercontent.com/pjreddie/darknet/master/cfg/yolov3.cfg
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8342 (8.1K) [text/plain]
Saving to: ‘yolov3.cfg’

yolov3.cfg          100%[===================>]   8.15K  --.-KB/s    in 0s      



In [4]:
import numpy as np
import cv2 as cv
from motrackers.detectors import YOLOv3
from motrackers import CentroidTracker, CentroidKF_Tracker, SORT, IOUTracker
from motrackers.utils import draw_tracks
import ipywidgets as widgets

In [5]:
# download pre-trained Yolov3 model
!./examples/pretrained_models/yolo_weights/get_yolo.sh

--2021-11-26 14:07:35--  https://pjreddie.com/media/files/yolov3.weights
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248007048 (237M) [application/octet-stream]
Saving to: ‘yolov3.weights.1’

yolov3.weights.1    100%[===================>] 236.52M   108MB/s    in 2.2s    

2021-11-26 14:07:37 (108 MB/s) - ‘yolov3.weights.1’ saved [248007048/248007048]

--2021-11-26 14:07:37--  https://raw.githubusercontent.com/pjreddie/darknet/master/cfg/yolov3.cfg
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8342 (8.1K) [text/plain]
Saving to: ‘yolov3.cfg.1’

yolov3.cfg.1        100%[===================>]   8.15K  --.-KB/s    in 0s  

In [6]:
# parameters
VIDEO_FILE = "/mydrive/video3.mp4"
WEIGHTS_PATH = './yolov3.weights'
CONFIG_FILE_PATH = './yolov3.cfg'
LABELS_PATH = "./examples/pretrained_models/yolo_weights/coco_names.json"

CONFIDENCE_THRESHOLD = 0.5
NMS_THRESHOLD = 0.2
DRAW_BOUNDING_BOXES = True
USE_GPU = False

In [ ]:
# chosen_tracker = widgets.Select(
#     options=["CentroidTracker", "CentroidKF_Tracker", "SORT", "IOUTracker"],
#     value='CentroidTracker',
#     rows=5,
#     description='MOTracker:',
#     disabled=False
# )
# chosen_tracker

In [7]:
tracker = SORT(max_lost=3, tracker_output_format='mot_challenge', iou_threshold=0.3)

In [8]:
model = YOLOv3(
    weights_path=WEIGHTS_PATH,
    configfile_path=CONFIG_FILE_PATH,
    labels_path=LABELS_PATH,
    confidence_threshold=CONFIDENCE_THRESHOLD,
    nms_threshold=NMS_THRESHOLD,
    draw_bboxes=DRAW_BOUNDING_BOXES,
    use_gpu=USE_GPU
)

### Working with images from 2 datasets in Kaggle (incomplete)

In [ ]:
imagefile = ...
cap = cv.VideoCapture(VIDEO_FILE)

### Working with video sequences to track humans

In [14]:
from google.colab.patches import cv2_imshow

def main(video_path, model, tracker):
    cap = cv.VideoCapture(video_path)

    # Check if camera opened successfully
    if (cap.isOpened() == False):
      print("Unable to read camera feed")

    # fourcc = cv.VideoWriter_fourcc(*'MP4V') # TIVX/DIVX for avi format
    file_path = '/mydrive/output2.mp4'
    fourcc = cv.VideoWriter_fourcc('M','J','P','G') # ('M','P,'E','G')
    fps = 50
    frame_width = int(cap.get(3))
    frame_height = int(cap.get(4))
    out = cv.VideoWriter(file_path, fourcc, 10, (frame_width, frame_height))

    # while True:
    for i in range(50):
        ok, image = cap.read()

        if not ok:
            print("Cannot read the video feed.")
            break

        image = cv.resize(image, (frame_width, frame_height))

        bboxes, confidences, class_ids = model.detect(image)
        
        tracks = tracker.update(bboxes, confidences, class_ids)
        
        updated_image = model.draw_bboxes(image.copy(), bboxes, confidences, class_ids)

        updated_image = draw_tracks(updated_image, tracks)
  
        out.write(updated_image)

        # cv2_imshow(updated_image) # to show the frame in Colab

        if cv.waitKey(1) & 0xFF == ord('q'):
            break

    # releasing everything at the end
    cap.release()
    out.release()
    cv.destroyAllWindows()

In [15]:
main(VIDEO_FILE, model, tracker)